# 稳定码模拟器

*版权所有 (c) 2023 百度量子计算研究所，保留所有权利。*

量子纠错是一组用于保护量子信息免受由于不完美的硬件或环境噪声引起的错误的技术。量子纠错的主要思想是以一种方式编码量子信息，使得可以检测和纠正错误而不破坏量子状态。这个概念对于实用量子计算机的发展至关重要，因为它们需要纠错来保持量子计算的完整性。

稳定码是一个庞大的量子纠错码家族，包括著名的比特翻转码、相位翻转码、Shor 码、Steane 码和表面码等。在本教程中，我们介绍了一种在[百度量子平台](https://quantum.baidu.com/)上模拟通用稳定码的系统方法。这种方法使用户能够研究各种类型噪声对纠错码性能的影响、评估不同的纠错码的性能、并评估量子算法对错误的鲁棒性。

## 量子纠错理论

### 理论简介

稳定码理论最初由 Daniel Gottesman 于 1997 年提出，已成为量子纠错理论和实践中的重要工具。稳定码背后的基本思想是以一种方式编码量子状态，使得可以检测和纠正错误。这种编码的关键是一组称为“稳定器”（Stabilizer）的算符，它们定义为保持编码状态不变的对称性生成元组成的群。具体而言，稳定器是一组对易的泡利算符，包括恒等算符 $I$，$X$，$Y$ 和 $Z$ 四个算符。更准确地说，稳定码由一对子空间定义，即“码子空间”和“补充子空间”，它们是彼此的正交补空间。码子空间由稳定器的本征态张成，这些本征态具有本征值为 $+1$。补充子空间由稳定器的本征态张成，这些本征态具有本征值为 $-1$。稳定器本身是作用于码子空间中的量子比特上的 Pauli 算符的乘积，只在这些比特上表现为非平凡的作用。

要了解稳定码如何保护量子信息，我们考虑一个单量子比特由于环境噪声而出现错误的情况。这个错误可以用在受影响的量子比特上表现为非平凡作用的 Pauli 算符来表示。如果码子空间被设计成稳定器与这个错误算符对易，则可以通过测量稳定器的本征值来检测错误。具体而言，如果所有稳定器的本征值都是 $+1$，则没有发生错误。如果一个或多个稳定器的本征值为 $-1$，则发生了错误，必须对量子比特进行纠正。

数学上，一个稳定码可以被三个基本参数所描述：

$$
\llbracket n, k, d\rrbracket,
$$

其中 $n$ 是组成码空间的*物理量子比特*数，$k$ 是可以受到保护的*逻辑量子比特*数，$d$ 是稳定码的*最小距离*（简称码距）。码距定义为使一个有效的码字变成另一个码字所需的最少量子比特错误，也可以等价定义为或者将一个编码状态转换成另一个编码状态所需的最少量子比特错误。码距用于度量稳定码的纠错能力，更大的码距这意味着更强的纠错能力。需要注意的是，每个有用的稳定码必须满足 $n > k$，因为物理冗余对于在不干扰逻辑状态的情况下检测和纠正错误是必要的。令 $\mathbb{P}_n$ 为 $n$ 量子比特 Pauli 群，$\mathcal{S}\subseteq\mathbb{P}_n$ 为与稳定码 $\llbracket n, k, d\rrbracket$ 对应的稳定器生成元集合。根据定义，稳定器生成元数目必须满足 $r = \vert\mathcal{S}\vert = n - k$。

### 逻辑量子态

我们称一个 $n$ 量子比特的量子态 $\vert\psi\rangle$ 是一个 $\llbracket n, k, d\rrbracket$ 稳定码的有效*逻辑量子态*，也称为*码字态*，当且仅当对于每个 Pauli 算子 $P\in\mathcal{S}$，其中 $\mathcal{S}$ 是该纠错码的稳定器生成元，满足 $P\vert\psi\rangle=\vert\psi\rangle$。也就是说，状态 $\vert\psi\rangle$ 被 $P$ 稳定。所有有效的逻辑量子态形成一个维度为 $2^k$ 的编码子空间，允许我们编码 $k$ 个逻辑量子比特。通常，我们需要先确定编码子空间的逻辑基态。

逻辑量子态在量子纠错中非常重要，因为它编码了需要受到保护免受错误干扰的信息。通过测量稳定器生成元中的元素，我们可以检测和纠正编码的量子比特中发生的错误，而不必直接测量逻辑比特的状态本身。这点非常重要，因为直接测量量子态可能会扰动其状态，可能引入错误，破坏了纠错的目的。

**举例说明。** 下面我们介绍简单易懂的三量子比特翻转码 $\llbracket 3, 1, 1\rrbracket$。它使用三个物理量子比特编码一个逻辑量子比特，稳定器生成元为 {"IZZ", "ZZI"}。通过简单的计算我们可以证明量子态 $\vert 000 \rangle$ 和 $\vert 111 \rangle$ 都被 $\mathcal{S}$ 稳定，因此三量子比特翻转码的码空间 $\mathcal{C}$ 的维度为 $2$。它的码空间和补空间可以按如下方式分类：

$$
\begin{align}
\mathcal{C} &= {\rm span}\{\vert 000 \rangle, \vert 111 \rangle\}, \\
\mathcal{C}_0^\perp &= {\rm span}\{\vert 001 \rangle, \vert 110 \rangle\}, \\
\mathcal{C}_1^\perp &= {\rm span}\{\vert 010 \rangle, \vert 101 \rangle\}, \\
\mathcal{C}_2^\perp &= {\rm span}\{\vert 100 \rangle, \vert 011 \rangle\}.
\end{align}
$$

仔细对比码空间和补空间，我们可以发现每个单比特翻转错误 $X_0$，$X_1$ 和 $X_2$ 都会将码空间旋转到唯一的补空间，即对于任意量子态 $\vert\psi\rangle\in\mathcal{C}$，有 $X_i\vert\psi\rangle \in \mathcal{C}_i^\perp$。通过测量不会改变逻辑基态 $\vert 000 \rangle$ 和 $\vert 111 \rangle$ 的两个稳定器 $IZZ$ 和 $ZZI$，我们能够检测具体是哪个量子比特发生翻转，然后将受损的量子比特翻转回其正确的状态。这意味着三量子比特翻转码能够检测和纠正任意单量子比特翻转错误。

### 基本流程

![QEC](./figures/qec-framework-cn.png "图1：量子纠错码的基本工作流程。")

如上图所示，稳定码的基本工作流程包括四个关键步骤：编码（Encoding）、错误检测（Error Detection）、错误纠正（Error Correction）和解码（Decoding）。具体来说，

+ **编码：** 在编码步骤中，量子信息 $|\psi\rangle$ 被编码成代码空间中受稳定码保护的逻辑状态。通常使用一组实现量子纠错码的量子门来完成编码。编码后的状态是由受代码保护的逻辑量子基态的叠加构成。

+ **错误检测：** 在错误检测步骤中，我们对稳定码的稳定器生成元进行测量以检查是否发生了任何错误，这通常被称为稳定器测量。稳定器是一组保持彼此对易且仅对代码子空间产生非平凡作用的 Pauli 算符。稳定器的本征值被测量以确定是否发生了任何错误。如果所有稳定器的本征值都是 $+1$，则状态位于代码子空间中没有发生任何错误。如果任何一个稳定器的本征值为 $-1$，则表示量子比特出现错误导致逻辑量子态改变。请注意，在这一步骤中，我们需要干净的辅助量子比特来实现稳定器测量并记录错误码。

+ **错误纠正：** 在错误纠正步骤中，根据前一步中稳定器测量得到的错误码，我们应用一组量子门来纠正错误。这些量子门被设计成可以撤销错误的影响并将状态返回到码子空间，一般通过查表获得。

+ **解码：** 在解码步骤中，我们需要从受保护的逻辑态中提取原始量子态信息 $\vert\psi\rangle$。我们需要通过测量代码子空间中的一部分量子比特来实现解码，这将把系统量子态转变为基态之一。测量结果用于重构被编码的原始量子态。最直接的实现方式是反转第一步的编码过程以实现解码，但是这种实现往往不够高效。

从上面量子纠错码的基本工作流程可以看出，稳定码成功的关键因素之一是其*稳定器*的选择：我们必须仔细选择合适的稳定器集合以便检测和纠正错误的同时保留量子信息。

总而言之，稳定码是一种保护量子信息免受环境噪声干扰的强大工具。它们由一组称为稳定器的对易 Pauli 算子定义，这种编码方式可以检测和纠正量子错误。稳定码已经成为量子纠错理论和实践中的重要工具，在量子计算和通信中已经找到了许多应用。

## 量子纠错实践

在本节中，我们将详细演示如何使用 QEP 实现的量子纠错码模拟器来模拟一些常见的稳定码。我们将首先展示如何实例化三量子比特翻转码并调用其属性和方法。接下来，我们将使用著名的的五量子比特纠错码来保护单比特量子状态免受任意量子噪声的影响。

###  三量子比特翻转码

如上所述，三量子比特翻转码可以保护一个量子比特免受单个位翻转错误的影响。下面我们介绍在 QEP 实例化三量子比特翻转码的方法。

首先，我们导入一些必要的软件包：

In [ ]:
import copy

import QCompute
from qcompute_qep.correction import BasicCode, ColorTable, FiveQubitCode
import qcompute_qep.utils.circuit as circuit

利用 QEP 中的 `BasicCode` 类构造稳定码的时候，必须指定它的稳定器生成元和它能够检测的错误类型。该稳定码需要的物理量子比特数和支持的逻辑量子比特数均能从稳定器生成元中获得。通过上面的案例，我们指导三量子比特翻转码的稳定器生成元为 {"IZZ", "ZZI"}，而它所能检测和纠正的错误类型为单量子比特翻转错误。利用 QEP，我们可以通过如下方式构造三量子比特翻转码：

In [ ]:
bitflip_code = BasicCode(stabilizers=['IZZ', 'ZZI'],
                         error_types=['X'],
                         name='Bit-Flip Code')

我们为稳定码提供了多种十分有用的方法。比如，您可以通过下面的调用来查看稳定码的基本信息：

In [ ]:
print(bitflip_code)

同时，您也可以通过极其简单的调用方式来获得稳定码的编码、错误检测、错误纠正和解码的量子电路实现：

In [ ]:
bitflip_code.print_encode_circuit()       # Visualize the encoding circuit of the bit-flip code
bitflip_code.print_detect_circuit()       # Visualize the detecting circuit of the bit-flip code
bitflip_code.print_correct_circuit()      # Visualize the correcting circuit of the bit-flip code
bitflip_code.print_decode_circuit();      # Visualize the decoding circuit of the bit-flip code

进一步，您可以将这些量子电路组合起来获得对应稳定码的完整量子电路，这个功能十分有益于量子纠错码的演示：

In [ ]:
bitflip_code.print_encode_decode_circuit()
bitflip_code.print_detect_correct_circuit()
bitflip_code.print_encode_detect_correct_decode_circuit();

### 五量子比特纠错码

五量子比特纠错码是能够保护一个逻辑量子比特免受*任何*单比特错误影响的最小量子纠错码。它最初由 R. Laflamme，C. Miquel，J.P. Paz 和 W.H. Zurek 于 1996 年发明。五量子比特码的稳定器生成元为 {"XZZXI"，"IXZZX"，"XIXZZ"，"ZXIXZ"}，它可以检测和纠正到任意单量子比特错误。在 QEP 中，五量子比特纠错码是一个内置量子纠错码，您可以通过以下方式获取：

In [ ]:
fivequbit_code = FiveQubitCode()
print(fivequbit_code)

同时，您也可以使用常规方法——即指定稳定器生成元和可检测错误类型——来构建五量子比特纠错码：

In [ ]:
fivequbit_code = BasicCode(stabilizers=['XZZXI', 'IXZZX', 'XIXZZ', 'ZXIXZ'],
                           error_types=['X', 'Y', 'Z'],
                           name='Five-Qubit Code')

接下来，我们将应用五量子比特量子纠错码来保护单量子比特量子态免受任意单量子比特量子噪声的干扰。在展开详细内容之前，我们首先定义以下打印函数以备后用。

In [ ]:
def print_counts(counts: dict, n: int, k: int):
    r"""Print measurement outcomes by splitting different roles.
    """
    for key, val in counts.items():
        # Print ancilla qubits
        print("'", end="")
        if n - k > 0:
            print("{}".format(ColorTable.ANCILLA + key[0:n - k] + ColorTable.END), end='')
            print("{}".format(ColorTable.PHYSICAL + key[n - k:2 * (n - k)] + ColorTable.END), end='')
        print("{}".format(ColorTable.ORIGINAL + key[2 * (n - k):2 * n - k] + ColorTable.END), end='')
        print("': {}".format(val))

我们想要保护的单量子比特量子态由如下量子电路所刻画：

In [ ]:
qp = QCompute.QEnv()
qp.Q.createList(1)
QCompute.X(qp.Q[0])

理想情况下，该量子电路总是生成量子态 $\vert 1 \rangle$。该量子电路具体形式为：

In [ ]:
circuit.print_circuit(qp.circuit, colors={'red': [0]});

如果我们在标准基下测量该量子态 $8192$ 次（假设测量不含噪声），那么我们将获得 $8192$ 个测量结果 $1$。利用 QEP 的本地模拟器，我们可以正确验证该结论：

In [ ]:
raw_qp = copy.deepcopy(qp)
QCompute.MeasureZ(*raw_qp.Q.toListPair())
raw_qp.backend(QCompute.BackendName.LocalBaiduSim2)
result = raw_qp.commit(shots=8192, fetchMeasure=True)
print_counts(result["counts"], n=1, k=1)

现在假设这个量子比特遭受了*完全退极化噪声*，这种噪声会将任意的输入状态变为最大混合态。此时，如果我们在计算基中测量该量子态 $8192$ 次，我们将按照均匀分布获得测量结果 $0$ 和 $1$。同样的，利用 QEP 本地模拟器，我们可以正确验证该结论：

In [ ]:
corrupted_qp = copy.deepcopy(qp)
# The qubit suffers from the completely depolarizing noise
QCompute.ID(corrupted_qp.Q[0])
corrupted_qp.noise(gateNameList=['ID'], noiseList=[QCompute.Depolarizing(bits=1, probability=1)], qRegList=[0])
QCompute.MeasureZ(*corrupted_qp.Q.toListPair())
corrupted_qp.backend(QCompute.BackendName.LocalBaiduSim2)
result = corrupted_qp.commit(shots=8192, fetchMeasure=True)
print_counts(result["counts"], n=1, k=1)

现在，我们使用五量子比特纠错码来保护该量子比特，使其免受完全退极化噪声的影响。具体流程为：我们先使用五量子比特纠错码来编码量子态，然后再让目标量子比特 Q0 遭受了完全退极化噪声，接着我们使用错误检测和纠正方法来消除该错误，最后我们通过解码获得正确量子态：

In [ ]:
# Step 1. Encode
enc_qp = fivequbit_code.encode(qp)
# Step 2. After encoding, the target qubit is damaged by the completely depolarizing noise
QCompute.ID(enc_qp.Q[0])
enc_qp.noise(gateNameList=['ID'], noiseList=[QCompute.Depolarizing(bits=1, probability=1)], qRegList=[0])
# Step 3. Detect and correct
cor_qp = fivequbit_code.detect_and_correct(enc_qp)
# Step 4. Decode
dec_qp = fivequbit_code.decode(cor_qp)

至此，`dec_qp` 对应的量子电路会生成正确量子态。为验证该结论，我们在标准基下测量这个量子纠错电路共 $8192$ 次，并观察和分析测量结果：

In [ ]:
QCompute.MeasureZ(*dec_qp.Q.toListPair())
counts = circuit.execute(qp=dec_qp, qc=QCompute.BackendName.LocalBaiduSim2, shots=8192)
print_counts(counts, n=fivequbit_code.n, k=fivequbit_code.k)

值得注意的是，尽管受到完全去极化噪声的影响，目标量子比特（用红色表示，Q0）始终产生正确的测量结果 $1$。这一显著结果凸显了五比特编码的有效性，它可以检测和纠正可能在物理量子比特中出现的任何单比特错误。另一方面，辅助比特（用黄色表示）会被破坏，它们的测量结果（一般被称为“错误码”）记录了发生的精确错误类型。

## 总结

在本教程中，我们简要介绍了稳定码的理论及其在[百度量子平台](https://quantum.baidu.com/)上的实现。我们希望这个模拟器能帮助用户研究各种类型噪声对纠错码性能的影响、评估不同纠错码的性能、以及评估量子算法对错误的鲁棒性。

## 参考资料

[1] Gottesman, Daniel. "Stabilizer Codes and Quantum Error Correction." PhD Thesis, **[California Institute of Technology](https://arxiv.org/abs/quant-ph/9705052)**, 1997.

[2] Nielsen, Michael A., and Isaac L. Chuang. "Quantum Computation and Quantum Information: 10th Anniversary Edition." **[Cambridge University Press](https://www.cambridge.org/highereducation/books/quantum-computation-and-quantum-information/01E10196D0A682A6AEFFEA52D53BE9AE#overview)**, 2010.

[3] Roffe, Joschka. "Quantum error correction: an introductory guide." **[Contemporary Physics](https://www.tandfonline.com/doi/abs/10.1080/00107514.2019.1667078)** 60.3 (2019): 226-245.

[4] Laflamme, Raymond, et al. "Perfect quantum error correcting code." **[Physical Review Letters](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.77.198)** 77.1 (1996): 198.